In [1]:
import numpy as np
import math
import nest
import nest.raster_plot
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib import rcParams 
import os.path


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.3
 Built: May  5 2022 07:35:57

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
# Variables
tones = [20,50,100,250,500,1000,2500,5000,10000,20000]
caps = [1,5,10,15,20]
angle = 0
max_ild = 2
time_sim = 1000 #us

In [3]:
# Fixed
w_head = 22 #cm
v_sound = 33000 #cm/s

ANFs2SBCs_weight = 2.0
ANFs2GBCs_weight = 1.0
GBCs2MNTBCs_weight = 16.0

SBCs2LSO_weight = 16.0
MNTBCs2LSO_weight = -2.0

SBCs2MSO_weight = 4.0
MNTBCs2MSO_weights = [0, -2.0, -4.0, -6.0, -8.0, -10.0, -12.0, -14.0, -16.0, -20.0]
n_battery = len(MNTBCs2MSO_weights)

delay_inh = 0.1

mean_rate = 0
mean_amplitude = 1000
noise_rate = 20

n_ANFs = 35000
ANFs2SBCs = 4
ANFs2GBCs = 20
SBCs2MSOs = int(ANFs2GBCs/ANFs2SBCs)
SBCs2LSOs = int(ANFs2GBCs/ANFs2SBCs)
n_SBCs = int(n_ANFs/ANFs2SBCs)
n_GBCs = int(n_ANFs/ANFs2GBCs)
n_MSOs = n_GBCs*n_battery

In [ ]:
for tone in tones:

    for C_m_bushy in caps:

        nest.ResetKernel()
        nest.local_num_threads = 16
        nest.resolution = 0.01 #0.01 # 10 us = minimum audible angle (MAA) increment of 1.25°

        freq = np.round(np.logspace(np.log(20),np.log(20000), num = 3500, base = np.exp(1)),2)
        channel_x = np.where(freq>=tone)[0][0]
        spectro = np.zeros((3500,time_sim))
        #amplitudes = np.round(stats.norm.pdf(np.linspace(-1, 1, 21) , 0, 1.0/(math.sqrt(2*math.pi)*1)),2)

        """"if(channel_x<10):
            spectro[channel_x:channel_x+10+1,:] = amplitudes[10:].reshape(11,1)*np.ones((11, time_sim))
            spectro[0:channel_x+1, :] = amplitudes[10-channel_x:11].reshape(channel_x+1,1)*np.ones((channel_x+1, time_sim))
        else:
            if(channel_x>3489):
                spectro[channel_x-10:channel_x+1] = amplitudes[:11].reshape(11,1)*np.ones((11, time_sim))
                spectro[channel_x:] = amplitudes[10:10+3500-channel_x].reshape(3500-channel_x,1)*np.ones((3500-channel_x, time_sim))
            else:
                spectro[channel_x - 10 : channel_x + 10 + 1, :] = amplitudes.reshape(21,1)*np.ones((21, time_sim))"""

        spectro[channel_x,:] = np.ones(time_sim)
        delta_x = (w_head*np.sin(np.deg2rad(angle)))
        itd = 1000*delta_x/v_sound #ms
        ipds = 2*np.pi*itd*freq/1000
        ild = angle*max_ild/90

        r_ANFs_amp = nest.Create('sinusoidal_poisson_generator',n_ANFs,
                params={'frequency': np.repeat(freq, 10),
                            'phase': np.repeat(np.rad2deg(ipds),10)}) #ITDs

        r_ANFs = nest.Create('parrot_neuron', n_ANFs)

        r_SBCs = nest.Create('iaf_cond_alpha', n_SBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        r_GBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        r_MNTBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        r_MSO = nest.Create('iaf_cond_alpha', n_MSOs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        r_LSO = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        l_ANFs_amp = nest.Create('sinusoidal_poisson_generator',n_ANFs,
                    params={'frequency': np.repeat(freq, 10),
                        'phase': 0})

        l_ANFs = nest.Create('parrot_neuron',n_ANFs)

        l_SBCs = nest.Create('iaf_cond_alpha', n_SBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        l_GBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        l_MNTBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        l_MSO = nest.Create('iaf_cond_alpha', n_MSOs, 
                         params = {'C_m': C_m_bushy, 'V_m': -70})

        l_LSO = nest.Create('iaf_cond_alpha', n_GBCs, 
                 params = {'C_m': C_m_bushy, 'V_m': -70})

        ANFs_noise = nest.Create('poisson_generator',1,
                         params = {'rate':noise_rate})

        s_rec_r = nest.Create('spike_recorder')
        s_rec_l = nest.Create('spike_recorder')

        nest.Connect(r_ANFs_amp, r_ANFs, 'one_to_one')
        nest.Connect(l_ANFs_amp, l_ANFs, 'one_to_one')

        nest.Connect(r_ANFs, s_rec_r, 'all_to_all')
        nest.Connect(l_ANFs, s_rec_l, 'all_to_all')

        nest.Connect(r_SBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_SBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_GBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_GBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_MNTBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_MNTBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_MSO, s_rec_r, 'all_to_all')
        nest.Connect(l_MSO, s_rec_l, 'all_to_all')

        nest.Connect(r_LSO, s_rec_r, 'all_to_all')
        nest.Connect(l_LSO, s_rec_l, 'all_to_all')

        for i in range(len(spectro)-1):
            if(np.any(spectro[i][:] > 0)):
                nest.Connect(ANFs_noise, r_ANFs[10*i:10*(i+1)], 'all_to_all')
                nest.Connect(ANFs_noise, l_ANFs[10*i:10*(i+1)], 'all_to_all')

        for i in range(n_SBCs):
            nest.Connect(r_ANFs[ANFs2SBCs*i:ANFs2SBCs*(i+1)], r_SBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2SBCs_weight})
            nest.Connect(l_ANFs[ANFs2SBCs*i:ANFs2SBCs*(i+1)], l_SBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2SBCs_weight})

        for i in range(n_GBCs):
            nest.Connect(r_ANFs[ANFs2GBCs*i:ANFs2GBCs*(i+1)], r_GBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2GBCs_weight})
            nest.Connect(l_ANFs[ANFs2GBCs*i:ANFs2GBCs*(i+1)], l_GBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2GBCs_weight})

        nest.Connect(r_GBCs, r_MNTBCs, 'one_to_one', syn_spec = {"weight":GBCs2MNTBCs_weight})
        nest.Connect(l_GBCs, l_MNTBCs, 'one_to_one', syn_spec = {"weight":GBCs2MNTBCs_weight})

        #MSO
        for i in range(n_GBCs):
            for j in range(n_battery):
                nest.Connect(r_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], r_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight})
                nest.Connect(l_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], r_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight}) 
                nest.Connect(r_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], l_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight})
                nest.Connect(l_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], l_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight}) 

        for i in range(n_GBCs):
            for j in range(n_battery):
                nest.Connect(l_MNTBCs[i], r_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":MNTBCs2MSO_weights[j], "delay": delay_inh})
                nest.Connect(r_MNTBCs[i], l_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":MNTBCs2MSO_weights[j], "delay": delay_inh})

        #LSO   
        for i in range(0, n_GBCs):
            nest.Connect(r_SBCs[SBCs2LSOs*i:SBCs2LSOs*(i+1)], r_LSO[i], 'all_to_all', syn_spec = {"weight":SBCs2LSO_weight})
            nest.Connect(l_SBCs[SBCs2LSOs*i:SBCs2LSOs*(i+1)], l_LSO[i], 'all_to_all', syn_spec = {"weight":SBCs2LSO_weight})   

        nest.Connect(r_MNTBCs, l_LSO, 'one_to_one', syn_spec = {"weight":MNTBCs2LSO_weight})
        nest.Connect(l_MNTBCs, r_LSO, 'one_to_one', syn_spec = {"weight":MNTBCs2LSO_weight})

        if(angle>=0): 
            mean_amplitude_r = mean_amplitude
            mean_amplitude_l = mean_amplitude/(10**(abs(ild)/20))
        else:
            mean_amplitude_l = mean_amplitude
            mean_amplitude_r = mean_amplitude/(10**(abs(ild)/20))

        def input_set_up(spectro, ms): 
            for r in range(0, len(spectro)-1):
                if spectro[r][ms] > 0:
                    r_ANFs_amp[10*r:10*(r+1)].set(amplitude = spectro[r][ms]*mean_amplitude_r)
                    l_ANFs_amp[10*r:10*(r+1)].set(amplitude = spectro[r][ms]*mean_amplitude_l)

        for i in range(time_sim):
            input_set_up(spectro,i)
            nest.Simulate(1)

        data_r = s_rec_r.get('events')
        data_l = s_rec_l.get('events')


        id_r_SBC1 = r_SBCs[0].get('global_id')
        id_r_GBC1 = r_GBCs[0].get('global_id')
        id_r_MNTBC1 = r_MNTBCs[0].get('global_id')
        id_r_MSO1 = r_MSO[0].get('global_id')
        id_r_LSO1 = r_LSO[0].get('global_id')

        id_l_SBC1 = l_SBCs[0].get('global_id')
        id_l_GBC1 = l_GBCs[0].get('global_id')
        id_l_MNTBC1 = l_MNTBCs[0].get('global_id')
        id_l_MSO1 = l_MSO[0].get('global_id')
        id_l_LSO1 = l_LSO[0].get('global_id')
        
        # BUSHY

        spikes_r_sbc = np.sort(data_r['times'][np.where((data_r['senders']>=id_r_SBC1) & (data_r['senders']<id_r_GBC1))])
        cells_r_sbc = np.unique(data_r['senders'][np.where((data_r['senders']>=id_r_SBC1) & (data_r['senders']<id_r_GBC1))], return_counts = True)
        
        spikes_r_gbc = np.sort(data_r['times'][np.where((data_r['senders']>=id_r_GBC1) & (data_r['senders']<id_r_MNTBC1))])        
        cells_r_gbc = np.unique(data_r['senders'][np.where((data_r['senders']>=id_r_GBC1) & (data_r['senders']<id_r_MNTBC1))], return_counts = True)
        
        spikes_l_sbc = np.sort(data_l['times'][np.where((data_l['senders']>=id_l_SBC1) & (data_l['senders']<id_l_GBC1))])
        cells_l_sbc = np.unique(data_l['senders'][np.where((data_l['senders']>=id_l_SBC1) & (data_l['senders']<id_l_GBC1))], return_counts = True)
        
        spikes_l_gbc = np.sort(data_l['times'][np.where((data_l['senders']>=id_l_GBC1) & (data_l['senders']<id_l_MNTBC1))])
        cells_l_gbc = np.unique(data_l['senders'][np.where((data_l['senders']>=id_l_GBC1) & (data_l['senders']<id_l_MNTBC1))], return_counts = True)

        #MSO

        #averaging on total number of active batteries --> result: battery with rates for each inh weight
        rate_r_MSO = np.zeros(n_battery)
        rate_l_MSO = np.zeros(n_battery)

        for i in range(int(n_MSOs/n_battery)): #n of batteries
            for j in range(n_battery): # neurons for battery
                if(id_r_MSO1+n_battery*i+j in data_r['senders']):
                    rate_r_MSO[j] += (np.unique(data_r['senders'][np.where(data_r['senders'] == id_r_MSO1+n_battery*i+j)], return_counts= True)[1][0])
                else:
                    rate_r_MSO[j] += 0
                if(id_l_MSO1+n_battery*i+j in data_l['senders']):
                    rate_l_MSO[j] += (np.unique(data_l['senders'][np.where(data_l['senders'] == id_l_MSO1+n_battery*i+j)], return_counts= True)[1][0])
                else:
                    rate_l_MSO[j] += 0     

        spikes_r_MSO = data_r['times'][np.where((data_r['senders']>=id_r_MSO1)&(data_r['senders']<id_r_LSO1))]
        cells_r_MSO = np.unique(data_r['senders'][np.where((data_r['senders']>=id_r_MSO1) & (data_r['senders']<id_r_LSO1))], return_counts = True)
        
        spikes_l_MSO = data_l['times'][np.where((data_l['senders']>=id_l_MSO1)&(data_l['senders']<id_l_LSO1))]
        cells_l_MSO = np.unique(data_l['senders'][np.where((data_l['senders']>=id_l_MSO1) & (data_l['senders']<id_l_LSO1))], return_counts = True)
        
        #LSO

        #averaging on total number of active cells --> result: averege rate of the population
        spikes_r_LSO = data_r['times'][np.where(data_r['senders']>=id_r_LSO1)]
        cells_r_LSO = np.unique(data_r['senders'][np.where(data_r['senders']>=id_r_LSO1)], return_counts = True)
        
        spikes_l_LSO = data_l['times'][np.where(data_l['senders']>=id_l_LSO1)]
        cells_l_LSO = np.unique(data_l['senders'][np.where(data_l['senders']>=id_l_LSO1)], return_counts = True)
        
        n_folder = 2
  
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_SBC_r_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_r_sbc)
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_GBC_r_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_r_gbc)

        
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_SBC_l_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_l_sbc)
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_GBC_l_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_l_gbc)
        
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/rates_MSO_r_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), rate_r_MSO)
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_MSO_r_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_r_MSO)
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_LSO_r_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_r_LSO)
        
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/rates_MSO_l_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), rate_l_MSO)
        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_MSO_l_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_l_MSO)

        np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results{}/spikes_LSO_l_freq_{}_C_{}'.format(n_folder, tone, C_m_bushy), spikes_l_LSO)


Jul 18 13:52:35 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.01 ms.

Jul 18 13:54:02 NodeManager::prepare_nodes [Info]: 
    Preparing 1253048 nodes for simulation.

Jul 18 13:54:03 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 1253048
    Simulation time (ms): 1
    Number of OpenMP threads: 16
    Number of MPI processes: 1

Jul 18 13:54:03 SimulationManager::run [Info]: 
    Simulation finished.

Jul 18 13:54:03 NodeManager::prepare_nodes [Info]: 
    Preparing 1253048 nodes for simulation.

Jul 18 13:54:03 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 1253048
    Simulation time (ms): 1
    Number of OpenMP threads: 16
    Number of MPI processes: 1

Jul 18 13:54:03 SimulationManager::run [Info]: 
    Simulation finished.

Jul 18 13:54:03 NodeManager::prepare_nodes [Info]: 
    Preparing 1253048 nodes for simulation.

Jul 18 13:54:03 SimulationManager::start_updating_ [Info]: 
    Number of l

In [ ]:
text_color = 'black' 
rcParams['text.color'] = text_color 
rcParams['axes.labelcolor'] = text_color 
rcParams['xtick.color'] = text_color 
rcParams['ytick.color'] = text_color 
plt.rc('font', size=16)          # controls default text sizes 
plt.rc('axes', titlesize=20)     # fontsize of the axes title 
plt.rc('axes', labelsize=18)    # fontsize of the x and y labels 
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels 
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels 
plt.rc('legend', fontsize=16)    # legend fontsize 
plt.rc('figure', titlesize=26)  # fontsize of the figure title

fig, axs = plt.subplots(1,2, figsize=(20,5))
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].imshow(spectro, aspect = 'auto')
axs[0].set_yticks([0, np.where(freq>=100)[0][0], np.where(freq>=1000)[0][0], np.where(freq==20000)[0][0]-10])
axs[0].set_yticklabels([20,100,1000,20000])


if(channel_x<10):
    axs[1].imshow(spectro[channel_x: channel_x + 20 + 1, :], aspect = 'auto')
    axs[1].set_yticks(np.arange(0,21,2))
    axs[1].set_yticklabels(freq[channel_x: channel_x + 20 + 1][::2])
else:
    if(channel_x>3489):
        axs[1].imshow(spectro[channel_x - 20: channel_x + 1, :], aspect = 'auto')
        axs[1].set_yticks(np.arange(0,21,2))
        axs[1].set_yticklabels(freq[channel_x - 20: channel_x + 1][::2])
    else:
        axs[1].imshow(spectro[channel_x - 10 : channel_x + 10 + 1, :], aspect = 'auto')
        axs[1].set_yticks(np.arange(0,21,2))
        axs[1].set_yticklabels(freq[channel_x - 10 : channel_x + 10 + 1][::2])



In [ ]:
spectro[channel_x+1,:]